Heat equation in one spatial dimension:
$$\frac{\partial T}{\partial t} = \alpha \frac{\partial ^2 T}{\partial x^2}$$

Where $\alpha$ is the thermal diffusivity and $T$ is the temperature.

Implicit works differently than what we used.  It will use more data from the "future" in the update including several values of $T$ at $t^{n+1}$.  It will make the scheme more difficult to apply.